In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# how to read a pickle file
tracks_features_processed = pd.read_pickle('tracks_features_processed.pkl')
track_identifier = pd.read_pickle('track_identifier.pkl')

In [ ]:
# tracks_features_processed.head(10)
track_identifier.head(10)

In [133]:
# playlist = pd.read_json('spotify_million_playlist_dataset/data/mpd.slice.0-999.json')
import json
# load data using Python JSON module
with open('spotify_million_playlist_dataset/data/mpd.slice.0-999.json','r') as f:
    data = json.loads(f.read())
    
# Normalizing data
df = pd.json_normalize(data, record_path =['playlists'])
print(df.columns)
df = df[['pid','name','tracks','num_tracks']]
df.head()

Index(['name', 'collaborative', 'pid', 'modified_at', 'num_tracks',
       'num_albums', 'num_followers', 'tracks', 'num_edits', 'duration_ms',
       'num_artists', 'description'],
      dtype='object')


,pid,name,tracks,num_tracks
0,0,Throwbacks,"[{'pos': 0, 'artist_name': 'Missy Elliott', 't...",52
1,1,Awesome Playlist,"[{'pos': 0, 'artist_name': 'Survivor', 'track_...",39
2,2,korean,"[{'pos': 0, 'artist_name': 'Hoody', 'track_uri...",64
3,3,mat,"[{'pos': 0, 'artist_name': 'Camille Saint-Saën...",126
4,4,90s,"[{'pos': 0, 'artist_name': 'The Smashing Pumpk...",17


In [134]:
df['tracks'].iloc[0][0]

{'pos': 0,
 'artist_name': 'Missy Elliott',
 'track_uri': 'spotify:track:0UaMYEvWZi0ZqiDOoHU3YI',
 'artist_uri': 'spotify:artist:2wIVse2owClT7go1WT98tk',
 'track_name': 'Lose Control (feat. Ciara & Fat Man Scoop)',
 'album_uri': 'spotify:album:6vV5UrXcfyQD1wu4Qo2I9K',
 'duration_ms': 226863,
 'album_name': 'The Cookbook'}

In [135]:
def convert_to_tracks(tracks_dict):
    tracks = []
    for track in tracks_dict:
        curr_artist = track['artist_name']
        curr_album = track['album_name']
        curr_album = re.sub(r'\[[^)]*\]', '', curr_album)
        curr_duration = track['duration_ms']
        curr_track = track['track_name']
        curr_track = re.sub(r'\[[^)]*\]', '', curr_track)
        # tmp_df = track_identifier[track_identifier['name'].str.contains(curr_track)].copy()
        tmp_df = track_identifier[track_identifier['album'].str.contains(curr_album)].copy()
        tmp_df = tmp_df[tmp_df['duration_ms'] < curr_duration + 100].copy()
        tmp_df = tmp_df[tmp_df['duration_ms'] > curr_duration - 100].copy()
        tmp_df = track_identifier[track_identifier['name'].str.contains(curr_track)].copy()
        # print(list(tmp_df['id']))
        if len(list(tmp_df['id'])) > 0:
            tracks.append(list(tmp_df['id'])[0])
    return tracks

def debug_convert_to_tracks(tracks_dict):
    tracks = []
    counter = 0
    for track in tracks_dict:
        print(counter)
        # print(track)
        curr_artist = track['artist_name']
        curr_album = track['album_name']
        curr_album = re.sub(r'\[[^)]*\]', '', curr_album)
        curr_duration = track['duration_ms']
        curr_track = track['track_name']
        curr_track = re.sub(r'\[[^)]*\]', '', curr_track)
        print(curr_track)
        print(curr_album)
        print(curr_artist)
        # tmp_df = track_identifier[track_identifier['name'].str.contains(curr_track)].copy()
        tmp_df = track_identifier[track_identifier['album'].str.contains(curr_album)].copy()
        # print(tmp_df)
        tmp_df = tmp_df[tmp_df['duration_ms'] < curr_duration + 100].copy()
        tmp_df = tmp_df[tmp_df['duration_ms'] > curr_duration - 100].copy()
        tmp_df = tmp_df[tmp_df['name'].str.contains(curr_track)].copy()
        print(tmp_df)
        # print(list(tmp_df['id']))
        if len(list(tmp_df['id'])) > 0:
            print('\ntrack to be added: ', list(tmp_df['id'])[0])
            tracks.append(list(tmp_df['id'])[0])
        counter += 1
    return tracks

In [136]:
for i in range(25):
    print(i)
    df['tracks'].iloc[i] = convert_to_tracks(df['tracks'].iloc[i])

0


/var/folders/jc/l8gc1q891rl5rcv8zg4yjywm0000gn/T/ipykernel_2022/3512832130.py:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  tmp_df = track_identifier[track_identifier['name'].str.contains(curr_track)].copy()
/var/folders/jc/l8gc1q891rl5rcv8zg4yjywm0000gn/T/ipykernel_2022/3512832130.py:11: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  tmp_df = track_identifier[track_identifier['album'].str.contains(curr_album)].copy()


1


/var/folders/jc/l8gc1q891rl5rcv8zg4yjywm0000gn/T/ipykernel_2022/1598124745.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tracks'].iloc[i] = convert_to_tracks(df['tracks'].iloc[i])


2
3
4
5
6
7
8
9
10


error: multiple repeat at position 19

In [137]:
df['num_tracks_id'] = df['tracks'].apply(lambda row: len(row))

In [138]:
df.head(20)

,pid,name,tracks,num_tracks,num_tracks_id
0,0,Throwbacks,"[6OANswAgi3wFWAOM3lpfsV, 03dhF72dTCeruDEf2W3G6...",52,39
1,1,Awesome Playlist,"[0aqYFk3KzREefAM1QdlyJe, 3OIHgTyQdiAGMmpjQaNxp...",39,29
2,2,korean,"[6dTwUqQFJRIqz6wh0ZWJfg, 2nMXOwuwxLITaCYvHQcPJ...",64,29
3,3,mat,"[6Z5YAVJ6StWeTutvpDGJ4O, 3ylZnu8g08EJF2b9ufLW6...",126,69
4,4,90s,"[0WAkoLKlEFRp4V3llA27KF, 0Q7zS5RqEbigi3dGkVKdh...",17,13
5,5,Wedding,"[5eZaT21ZVGyGHJ8kcwaNxA, 4Shbv9t1OjvuviwyZ5qk4...",80,61
6,6,I Put A Spell On You,"[7d8XI4XryEEn5PSEU0Qucw, 0orNm6qQT2B5WFi0l7l3P...",16,13
7,7,2017,"[3gQ50l7fiRPwdkz88w8oLz, 2f5w2zUXijp0eHafUZtJA...",53,35
8,8,BOP,"[6c5PYRjENFIsecZ08o6oDP, 0Y718MkWbqbbuGmdLkvO0...",46,29
9,9,old country,"[7jWbXvrgdbkajU8L28ahn5, 7jWbXvrgdbkajU8L28ahn...",21,12


In [139]:
df['tracks'].iloc[10] = debug_convert_to_tracks(df['tracks'].iloc[10])

0
Big Lie
Stoney
Post Malone
Empty DataFrame
Columns: [id, name, album, artists, duration_ms]
Index: []
1
All We Got (feat. Kanye West & Chicago Children's Choir)
Coloring Book
Chance The Rapper


/var/folders/jc/l8gc1q891rl5rcv8zg4yjywm0000gn/T/ipykernel_2022/3512832130.py:40: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  tmp_df = tmp_df[tmp_df['name'].str.contains(curr_track)].copy()


Empty DataFrame
Columns: [id, name, album, artists, duration_ms]
Index: []
2
Planez
Late Nights: The Album
Jeremih
Empty DataFrame
Columns: [id, name, album, artists, duration_ms]
Index: []
3
Rich As Fuck
I Am Not A Human Being II
Lil Wayne
Empty DataFrame
Columns: [id, name, album, artists, duration_ms]
Index: []
4
The Show Goes On
Lasers
Lupe Fiasco
Empty DataFrame
Columns: [id, name, album, artists, duration_ms]
Index: []
5
Tiimmy Turner
Tiimmy Turner
Desiigner
Empty DataFrame
Columns: [id, name, album, artists, duration_ms]
Index: []
6
Chillin
Attention Deficit
Wale
Empty DataFrame
Columns: [id, name, album, artists, duration_ms]
Index: []
7
T-Shirt
Culture
Migos
                            id     name    album  artists  duration_ms
638234  7KOlJ92bu51cltsD9KU5I7  T-Shirt  Culture  [Migos]       242407

track to be added:  7KOlJ92bu51cltsD9KU5I7
8
Portland
More Life
Drake
Empty DataFrame
Columns: [id, name, album, artists, duration_ms]
Index: []
9
Light
I Decided.
Big Sean
Empty Da

/var/folders/jc/l8gc1q891rl5rcv8zg4yjywm0000gn/T/ipykernel_2022/3512832130.py:40: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  tmp_df = tmp_df[tmp_df['name'].str.contains(curr_track)].copy()


Empty DataFrame
Columns: [id, name, album, artists, duration_ms]
Index: []
14
Say Something (Featuring Drake)
Shock Value II
Timbaland
Empty DataFrame
Columns: [id, name, album, artists, duration_ms]
Index: []
15
Studio
Oxymoron
ScHoolboy Q


...

/var/folders/jc/l8gc1q891rl5rcv8zg4yjywm0000gn/T/ipykernel_2022/3512832130.py:40: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  tmp_df = tmp_df[tmp_df['name'].str.contains(curr_track)].copy()


Empty DataFrame
Columns: [id, name, album, artists, duration_ms]
Index: []
38
Deja Vu
4 Your Eyez Only
J. Cole
Empty DataFrame
Columns: [id, name, album, artists, duration_ms]
Index: []
39
Straightjacket
The Story of Us
Quinn XCII
Empty DataFrame
Columns: [id, name, album, artists, duration_ms]
Index: []
40
White Walls (feat. ScHoolboy Q, Hollis)
The Heist 
Macklemore & Ryan Lewis
Empty DataFrame
Columns: [id, name, album, artists, duration_ms]
Index: []
41
Break The Bitch Down (feat. K. Camp)
Luca Brasi 2: Gangsta Grillz
Kevin Gates


/var/folders/jc/l8gc1q891rl5rcv8zg4yjywm0000gn/T/ipykernel_2022/3512832130.py:40: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  tmp_df = tmp_df[tmp_df['name'].str.contains(curr_track)].copy()
/var/folders/jc/l8gc1q891rl5rcv8zg4yjywm0000gn/T/ipykernel_2022/3512832130.py:40: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  tmp_df = tmp_df[tmp_df['name'].str.contains(curr_track)].copy()


Empty DataFrame
Columns: [id, name, album, artists, duration_ms]
Index: []
42
No Type
SremmLife
Rae Sremmurd
Empty DataFrame
Columns: [id, name, album, artists, duration_ms]
Index: []
43
D U Down
By Any Means 2
Kevin Gates
Empty DataFrame
Columns: [id, name, album, artists, duration_ms]
Index: []
44
Fuckin Right
By Any Means 2
Kevin Gates
Empty DataFrame
Columns: [id, name, album, artists, duration_ms]
Index: []
45
GOMD
By Any Means 2
Kevin Gates
Empty DataFrame
Columns: [id, name, album, artists, duration_ms]
Index: []
46
Hot N*gga
Hot N*gga
Bobby Shmurda
Empty DataFrame
Columns: [id, name, album, artists, duration_ms]
Index: []
47
Thinking With My D**k (feat. Juciy J)
Stranger Than Fiction
Kevin Gates


error: multiple repeat at position 19

In [130]:
debug1 = track_identifier[track_identifier['album'].str.contains('Stranger Than Fiction')].copy()

In [131]:
debug1

,id,name,album,artists,duration_ms
355730,4UpDqQa9NV2J5V1sJdgYHW,Coming Of The Storm,Stranger Than Fiction (Deluxe Edition),[Forté],191200
355731,3OXDYWZpucDf2VZIQpWAxK,The Inner Circle,Stranger Than Fiction (Deluxe Edition),[Forté],215893
355732,0FLpANV45QGJwJujJUP6eX,Stranger Than Fiction,Stranger Than Fiction (Deluxe Edition),[Forté],209840
355733,3xMw0S5XRMzPGoLRBGZzDZ,G 13 - Devoid Of Thought,Stranger Than Fiction (Deluxe Edition),[Forté],72467
355734,1ubOlz7OJ2HU0i5vDN3fTU,Mein Madness,Stranger Than Fiction (Deluxe Edition),[Forté],235133
355735,6dio6OsHrBKHL2wTiM33S8,Time And Time Again,Stranger Than Fiction (Deluxe Edition),[Forté],209800
355736,3Zj3HzbsYVk6a7Hv6Zv15n,Digitator,Stranger Than Fiction (Deluxe Edition),[Forté],264760
355737,6CxnI6gpY8ol1uoh3z3y0q,Between The Lies,Stranger Than Fiction (Deluxe Edition),[Forté],156107
355738,14r7590zLUSanwxiPok69t,That Last Word,Stranger Than Fiction (Deluxe Edition),[Forté],207000
355739,62jAVqBWTmZqttpwnWJgDP,The Promise,Stranger Than Fiction (Deluxe Edition),[Forté],276227
